# Dynamical Matrix Representation

\begin{equation}
V = \frac{1}{2} \sum_{\alpha,\beta}^{3,3} u_{\alpha} C_{\alpha \beta} u_{\beta}
\end{equation}

\begin{equation}
C_{\alpha \beta} = \frac{\partial^2 V}{\partial u_\alpha \partial u_\beta}
\end{equation}

\begin{equation}
D_{\alpha \beta}(\mathbf{k}) = \frac{1}{\sqrt{m_A m_B}} \sum_{\mathbf{r}_A,\mathbf{r}_B} C_{\alpha \beta} e^{i \mathbf{k} \cdot (\mathbf{r}_A - \mathbf{r}_B)}
\end{equation}

In [1]:
import numpy as np
import seekpath
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, FloatSlider
import cmath

def solver(C1,C2,mass,a):
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=(5,5))
    
    ax.tick_params(axis='both',direction='in')
    plt.ylabel('Frequency')
    
    structure = ([[a,0,0],[0,a,0],[0,0,a]],[[0,0,0]],[1])

    k_path = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_abs']
    k_ticks = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_linearcoord']
    k_labels = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_labels']
    k_segments = seekpath.get_explicit_k_path(structure=structure)['explicit_segments']
    
    
    lat = structure[0]
    position = structure[1]
    
    G = np.zeros((3, 3))
    G[0] = 2* np.pi * np.cross(lat[1], lat[2]) / (np.inner(
            lat[0], np.cross(lat[1], lat[2])))
    G[1] = 2* np.pi * np.cross(lat[2], lat[0]) / (np.inner(
            lat[1], np.cross(lat[2], lat[0])))
    G[2] =  2* np.pi * np.cross(lat[0], lat[1]) / (np.inner(
            lat[2], np.cross(lat[0], lat[1])))
    
    kpath = np.inner(G,k_path/(2*np.pi)).T

    
    #Find nearest neighbors and next nearest neighbors from a set of translations.
    R = []
    for dimx in range(-4,5):
        for dimy in range(-4,5):
            for dimz in range(-4,5):
                R1 = [x * dimx for x in lat[0]]
                R2 = [y * dimy for y in lat[1]]
                R3 = [z * dimz for z in lat[2]]
                R.append([sum(x) for x in zip(R1,R2,R3)])
    
    dist = []
    index = []
    for i in range(len(R)):          
        norm = np.linalg.norm(np.asarray(position)-np.asarray(R[i]))
        dist.append(norm)

    for i in range(len(dist)):
        if dist[i] <= sorted(dist)[7]:
            index.append(i)
      
    NN = [R[i] for i in index]
    
    #arguments for the geometric exponential used in the dynamical matrix.
    dis = position + np.inner(NN,lat)

    #lattice_potential = np.arange(3).reshape(3)
    
    dynamical_matrix = np.arange(3**2*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    frequency = np.arange(3*len(k_ticks),dtype=complex).reshape(3,len(k_ticks))
    normal_modes = np.arange(3**2*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    
    elastic_const = [[C1,0,0],[0,C1,0],[0,0,C1]]
    #EC2 = [[C2,C2,C2],[C2,C2,C2],[C2,C2,C2]]
    
    for k in range(len(kpath)):
        for idynm in range(3):
            for jdynm in range(3):
                elastic = elastic_const[idynm][jdynm]
                
                if idynm == jdynm:
                    val = 2
                #else:
                #    val = 0
                    
                for Rn in range(len(dis)):
                   
                    if idynm == jdynm:
                        #Geometric exponent for the nearest neighbors. 
                        if idynm == 1:

                            if np.inner(dis[Rn],lat[jdynm]) != 0 and np.inner(dis[Rn],[sum(x) for x in zip(lat[abs(idynm-3)],lat[abs(idynm-1)])]) == 0:
                                val -= cmath.exp(1j*np.dot(kpath[k],dis[Rn]))
                        
                        else:
                            if np.inner(dis[Rn],lat[jdynm]) != 0 and np.inner(dis[Rn],[sum(x) for x in zip(lat[abs(idynm-2)],lat[abs(idynm-1)])]) == 0:
                                val -= cmath.exp(1j*np.dot(kpath[k],dis[Rn]))
                                
                    #else:
                    #    if np.inner(dis[Rn],lat[idynm]) != 0 and np.inner(dis[Rn],lat[jdynm]) != 0:   
                    #        val += cmath.exp(1j*np.dot(kpath[k],dis[Rn]))
                
                #populate the matrix.
                dynamical_matrix[idynm][jdynm][k] = (1/mass) * elastic * (val)
        #ensure that the matrix is hermitian.
        dynamical_matrix[:,:,k] = 1/2*(dynamical_matrix[:,:,k] + dynamical_matrix[:,:,k].conj().T)
        #diagonalization.
        frequency[:,k], normal_modes[:,:,k] = np.linalg.eigh(dynamical_matrix[:,:,k])[:]
    
    #plotting.
    for i in range(3):
        plt.plot(k_ticks,np.sqrt(abs(frequency[i,:])))
    
    plt.xlim(0,max(k_ticks))
    plt.ylim(0)
    
    for i in range(len(k_segments[:])):
        if k_segments[i-1][1] == k_segments[i][0]:
            k_labels[k_segments[i-1][1]] = k_labels[k_segments[i][0]-1] + '|' + k_labels[k_segments[i][0]]
            k_labels[k_segments[i][0]-1] = ''
            plt.axvline(x=k_ticks[k_segments[i][0]],color='black')
        else:
            pass
    
    for i in range(len(k_labels)):
        if k_labels[i] == "GAMMA":
            k_labels[i] = "\Gamma"
        else:
            pass

    k_labels = [str("$" + latx + "$") for latx in k_labels]
    
    plt.xticks([k_ticks[i] for i,x in enumerate(k_labels) if x != '$$'],[k_labels[i] for i,x in enumerate(k_labels) if x != '$$'])
    
    plt.show()

interactive_plot = interact(solver, C1=FloatSlider(value=2.0,min=0., max=4, step=0.1,description='C1:'),
                            C2=FloatSlider(value=0.0,min=0., max=4, step=0.1,description='C2:'),
                            mass=FloatSlider(value=1.0,min=1, max=4, step=0.1,description='mass:'),
                            a=FloatSlider(value=1.5,min=0.5, max=4, step=0.05,description='a:'))
interactive_plot

interactive(children=(FloatSlider(value=2.0, description='C1:', max=4.0), FloatSlider(value=0.0, description='…

<function __main__.solver(C1, C2, mass, a)>